In [1]:
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkFiles
from pyspark.ml.linalg import DenseVector
from sklearn.metrics import accuracy_score
import pandas as pd


import pyspark
from pyspark import SparkContext


import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import tree
from sklearn.model_selection import train_test_split
from io import StringIO
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report,recall_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier



from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.semi_supervised import LabelSpreading
from sklearn.svm import LinearSVC
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier


from sklearn.metrics import roc_curve,precision_recall_curve
from sklearn.metrics import roc_auc_score,confusion_matrix

/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
sc =SparkContext()
sc.addFile("H-1B_Disclosure_Data_FY2019.csv")

In [3]:
sqlContext = SQLContext(sc)
sqlContext
df = sqlContext.read.csv(SparkFiles.get("H-1B_Disclosure_Data_FY2019.csv"), header=True, inferSchema= True)
df.printSchema()

root
 |-- CASE_NUMBER: string (nullable = true)
 |-- CASE_STATUS: string (nullable = true)
 |-- CASE_SUBMITTED: string (nullable = true)
 |-- DECISION_DATE: string (nullable = true)
 |-- ORIGINAL_CERT_DATE: string (nullable = true)
 |-- VISA_CLASS: string (nullable = true)
 |-- EMPLOYMENT_START_DATE: string (nullable = true)
 |-- EMPLOYMENT_END_DATE: string (nullable = true)
 |-- EMPLOYER_NAME: string (nullable = true)
 |-- EMPLOYER_BUSINESS_DBA: string (nullable = true)
 |-- EMPLOYER_ADDRESS: string (nullable = true)
 |-- EMPLOYER_CITY: string (nullable = true)
 |-- EMPLOYER_STATE: string (nullable = true)
 |-- EMPLOYER_POSTAL_CODE: string (nullable = true)
 |-- EMPLOYER_COUNTRY: string (nullable = true)
 |-- EMPLOYER_PROVINCE: string (nullable = true)
 |-- EMPLOYER_PHONE: string (nullable = true)
 |-- EMPLOYER_PHONE_EXT: string (nullable = true)
 |-- SECONDARY_ENTITY: string (nullable = true)
 |-- SECONDARY_ENTITY_BUSINESS_NAME: string (nullable = true)
 |-- AGENT_REPRESENTING_EMPLOY

In [4]:

cols=['CASE_NUMBER','DECISION_DATE','ORIGINAL_CERT_DATE','EMPLOYER_BUSINESS_DBA','EMPLOYER_ADDRESS','EMPLOYER_CITY']
col2=['EMPLOYER_PROVINCE','EMPLOYER_PHONE','EMPLOYER_PHONE_EXT','SECONDARY_ENTITY','SECONDARY_ENTITY_BUSINESS_NAME']
col3=['SECONDARY_ENTITY', 'SECONDARY_ENTITY_BUSINESS_NAME', 'AGENT_REPRESENTING_EMPLOYER', 'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE']
col4=['SECONDARY_ENTITY', 'SECONDARY_ENTITY_BUSINESS_NAME', 'AGENT_REPRESENTING_EMPLOYER', 'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE']
col5=['VISA_CLASS','EMPLOYER_COUNTRY','WAGE_RATE_OF_PAY_FROM','WAGE_UNIT_OF_PAY','PW_SOURCE','PW_SOURCE_OTHER','PW_WAGE_LEVEL','WORKSITE_POSTAL_CODE','WORKSITE_CITY','WAGE_RATE_OF_PAY_TO','JOB_TITLE','SOC_NAME','CASE_SUBMITTED','WORKSITE_POSTAL_CODE','STATUTORY_BASIS','EMPLOYER_POSTAL_CODE','APPX_A_NO_OF_EXEMPT_WORKER_1','APPX_A_NAME_OF_INSTITUTION_1','APPX_A_FIELD_OF_STUDY_1', 'APPX_A_DATE_OF_DEGREE_1', 'APPX_A_NO_OF_EXEMPT_WORKER_2', 'APPX_A_NAME_OF_INSTITUTION_2', 'APPX_A_FIELD_OF_STUDY_2', 'APPX_A_DATE_OF_DEGREE_2', 'APPX_A_NO_OF_EXEMPT_WORKER_3', 'APPX_A_NAME_OF_INSTITUTION_3', 'APPX_A_FIELD_OF_STUDY_3', 'APPX_A_DATE_OF_DEGREE_3', 'APPX_A_NO_OF_EXEMPT_WORKER_4', 'APPX_A_NAME_OF_INSTITUTION_4', 'APPX_A_FIELD_OF_STUDY_4', 'APPX_A_DATE_OF_DEGREE_4', 'APPX_A_NO_OF_EXEMPT_WORKER_5', 'APPX_A_NAME_OF_INSTITUTION_5', 'APPX_A_FIELD_OF_STUDY_5', 'APPX_A_DATE_OF_DEGREE_5','MASTERS_EXEMPTION']

df2=df.drop(*cols)
df2=df2.drop(*col2)
df2=df2.drop(*col3)
df2=df2.drop(*col4)
df2=df2.drop(*col5)

print(df2.printSchema())

['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE', 'ORIGINAL_CERT_DATE', 'VISA_CLASS', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE', 'EMPLOYER_NAME', 'EMPLOYER_BUSINESS_DBA', 'EMPLOYER_ADDRESS', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT', 'SECONDARY_ENTITY', 'SECONDARY_ENTITY_BUSINESS_NAME', 'AGENT_REPRESENTING_EMPLOYER', 'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE', 'JOB_TITLE', 'SOC_CODE', 'SOC_NAME', 'NAICS_CODE', 'TOTAL_WORKERS', 'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT', 'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION', 'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_OTHER', 'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY', 'H1B_DEPENDENT', 'WORKSITE_CITY', 'WORKSITE_COUNTY', 'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE', 'WILLFUL_VIOLA

In [5]:
pd_df=df2.toPandas()


In [7]:
set(pd_df.H1B_DEPENDENT)

{'N', None, 'Y'}

In [8]:
pd_df.isnull().sum(axis=0)

CASE_STATUS                        0
EMPLOYMENT_START_DATE              0
EMPLOYMENT_END_DATE                0
EMPLOYER_NAME                      1
EMPLOYER_STATE                    41
SOC_CODE                           4
NAICS_CODE                         2
TOTAL_WORKERS                      0
NEW_EMPLOYMENT                     0
CONTINUED_EMPLOYMENT               0
CHANGE_PREVIOUS_EMPLOYMENT         0
NEW_CONCURRENT_EMPLOYMENT          0
CHANGE_EMPLOYER                    0
AMENDED_PETITION                   0
FULL_TIME_POSITION                 0
PREVAILING_WAGE                83662
PW_UNIT_OF_PAY                 83662
H1B_DEPENDENT                  15074
WORKSITE_COUNTY                   46
WORKSITE_STATE                    18
WILLFUL_VIOLATOR               15079
SUPPORT_H1B                   378454
dtype: int64

In [9]:
pd_df['WILLFUL_VIOLATOR']=pd_df['WILLFUL_VIOLATOR'].fillna('N')
pd_df['SUPPORT_H1B']=pd_df['SUPPORT_H1B'].fillna('N')

pd_df['SUPPORT_H1B']=pd_df['SUPPORT_H1B'].replace('NA','N')

pd_df['SUPPORT_H1B']=pd_df['SUPPORT_H1B'].replace('N',0)

pd_df['SUPPORT_H1B']=pd_df['SUPPORT_H1B'].replace('Y',1)

pd_df['WILLFUL_VIOLATOR']=pd_df['WILLFUL_VIOLATOR'].replace('N',0)

pd_df['WILLFUL_VIOLATOR']=pd_df['WILLFUL_VIOLATOR'].replace('Y',1)



pd_df['H1B_DEPENDENT']=pd_df['H1B_DEPENDENT'].fillna('N')

pd_df['H1B_DEPENDENT']=pd_df['H1B_DEPENDENT'].replace('NA','N')

pd_df['H1B_DEPENDENT']=pd_df['H1B_DEPENDENT'].replace('N',0)

pd_df['H1B_DEPENDENT']=pd_df['H1B_DEPENDENT'].replace('Y',1)


pd_df['H1B_DEPENDENT']=pd_df['H1B_DEPENDENT'].fillna('N')

In [10]:
pd_df=pd_df.dropna()

In [11]:
pd_df.isnull().sum(axis=0)

CASE_STATUS                   0
EMPLOYMENT_START_DATE         0
EMPLOYMENT_END_DATE           0
EMPLOYER_NAME                 0
EMPLOYER_STATE                0
SOC_CODE                      0
NAICS_CODE                    0
TOTAL_WORKERS                 0
NEW_EMPLOYMENT                0
CONTINUED_EMPLOYMENT          0
CHANGE_PREVIOUS_EMPLOYMENT    0
NEW_CONCURRENT_EMPLOYMENT     0
CHANGE_EMPLOYER               0
AMENDED_PETITION              0
FULL_TIME_POSITION            0
PREVAILING_WAGE               0
PW_UNIT_OF_PAY                0
H1B_DEPENDENT                 0
WORKSITE_COUNTY               0
WORKSITE_STATE                0
WILLFUL_VIOLATOR              0
SUPPORT_H1B                   0
dtype: int64

In [12]:
import seaborn as sns
import matplotlib.pyplot as plt

corr = pd_df.corr()
corr

,NAICS_CODE,TOTAL_WORKERS,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B
NAICS_CODE,1.000000,-0.098190,-0.086696,-0.032597,-0.023386,-0.097908,0.194140,0.004784,0.192441
TOTAL_WORKERS,-0.098190,1.000000,0.583633,0.247455,0.498143,0.621934,-0.035097,-0.001890,-0.035476
CHANGE_PREVIOUS_EMPLOYMENT,-0.086696,0.583633,1.000000,0.122062,0.377058,0.525901,-0.059081,-0.002459,-0.058412
NEW_CONCURRENT_EMPLOYMENT,-0.032597,0.247455,0.122062,1.000000,0.167433,0.173050,-0.018067,-0.000373,-0.017774
CHANGE_EMPLOYER,-0.023386,0.498143,0.377058,0.167433,1.000000,0.526877,0.024138,-0.000841,0.024569
AMENDED_PETITION,-0.097908,0.621934,0.525901,0.173050,0.526877,1.000000,0.023604,0.000222,0.023246
H1B_DEPENDENT,0.194140,-0.035097,-0.059081,-0.018067,0.024138,0.023604,1.000000,0.022600,0.994309
WILLFUL_VIOLATOR,0.004784,-0.001890,-0.002459,-0.000373,-0.000841,0.000222,0.022600,1.000000,0.025318
SUPPORT_H1B,0.192441,-0.035476,-0.058412,-0.017774,0.024569,0.023246,0.994309,0.025318,1.000000


In [14]:
data_preprocess=pd_df[['PREVAILING_WAGE','PW_UNIT_OF_PAY']]

In [15]:
data_preprocess.PREVAILING_WAGE.replace('$','')

data_preprocess.PREVAILING_WAGE=data_preprocess.PREVAILING_WAGE.apply(lambda x: x.replace('$',''))
data_preprocess.PREVAILING_WAGE=data_preprocess.PREVAILING_WAGE.apply(lambda x: x.replace(',',''))

/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [16]:
for i in data_preprocess:
    print(i)

PREVAILING_WAGE
PW_UNIT_OF_PAY


In [17]:
set(data_preprocess.PW_UNIT_OF_PAY)

{'Bi-Weekly', 'Hour', 'Month', 'Week', 'Year'}

In [18]:
z=[]
for index,row in data_preprocess.iterrows():
    if row.PW_UNIT_OF_PAY=='Year':
        z.append(1)
    elif row.PW_UNIT_OF_PAY=='Hour':
        z.append(1920)
    elif row.PW_UNIT_OF_PAY=='Month':
        z.append(12)
    elif row.PW_UNIT_OF_PAY=='Week':
        z.append(48)
    elif row.PW_UNIT_OF_PAY=='Bi-Weekly':
        z.append(96)
    else:
        z.append(0)
    
        
        



In [19]:
data_preprocess['multiplier']=z

/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
data_preprocess['PREVAILING_WAGE_int']=pd.to_numeric(data_preprocess.PREVAILING_WAGE)
data_preprocess['new_wage_in_dollar_per_year']=(data_preprocess.PREVAILING_WAGE_int)*data_preprocess.multiplier

/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
pd_df['TOTAL_WAGE']=data_preprocess.new_wage_in_dollar_per_year

In [23]:
pd_df.columns

Index(['CASE_STATUS', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE',
       'EMPLOYER_NAME', 'EMPLOYER_STATE', 'SOC_CODE', 'NAICS_CODE',
       'TOTAL_WORKERS', 'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT',
       'CHANGE_PREVIOUS_EMPLOYMENT', 'NEW_CONCURRENT_EMPLOYMENT',
       'CHANGE_EMPLOYER', 'AMENDED_PETITION', 'FULL_TIME_POSITION',
       'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'H1B_DEPENDENT', 'WORKSITE_COUNTY',
       'WORKSITE_STATE', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'TOTAL_WAGE'],
      dtype='object')

In [24]:


pd_df2 = pd_df.drop(pd_df[(pd_df.CASE_STATUS =='CERTIFIED-WITHDRAWN')].index)
pd_df2 = pd_df2.drop(pd_df2[(pd_df2.CASE_STATUS =='WITHDRAWN')].index)



{'CERTIFIED', 'DENIED'}

In [25]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
x=pd_df2
for i in x:
    x[i]=le.fit_transform(x[i])

In [26]:
set(x.CASE_STATUS)

{0, 1}

In [27]:
x2=x.drop(['CASE_STATUS', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'WORKSITE_COUNTY',
       'WORKSITE_STATE','EMPLOYMENT_START_DATE','EMPLOYMENT_END_DATE'], axis=1)

x2=x2.drop([ 'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT',
       'CHANGE_PREVIOUS_EMPLOYMENT', 'NEW_CONCURRENT_EMPLOYMENT',
       'CHANGE_EMPLOYER', 'AMENDED_PETITION','SUPPORT_H1B'], axis=1)


y=x.CASE_STATUS




,CASE_STATUS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_STATE,SOC_CODE,NAICS_CODE,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,...,AMENDED_PETITION,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,H1B_DEPENDENT,WORKSITE_COUNTY,WORKSITE_STATE,WILLFUL_VIOLATOR,SUPPORT_H1B,TOTAL_WAGE
CASE_STATUS,1.000000,-0.015938,-0.011873,0.006379,-0.009927,0.042184,-0.026265,0.005182,0.003979,-0.003403,...,-0.005500,-0.037473,-0.010189,-0.034091,-0.045161,0.000845,0.000077,0.004322,-0.045116,-0.032845
EMPLOYMENT_START_DATE,-0.015938,1.000000,0.866347,0.009805,-0.019622,0.013479,-0.010785,0.003735,0.094475,-0.066523,...,-0.091519,-0.035266,0.068744,-0.046231,-0.036711,0.002641,-0.008945,-0.004027,-0.036100,-0.125136
EMPLOYMENT_END_DATE,-0.011873,0.866347,1.000000,0.018228,-0.027102,0.024115,-0.006224,-0.004092,0.091119,-0.067342,...,-0.098456,-0.043737,0.066260,-0.052975,-0.052550,0.004573,-0.012891,-0.004034,-0.052072,-0.133848
EMPLOYER_NAME,0.006379,0.009805,0.018228,1.000000,-0.110785,0.077274,0.126356,-0.020635,0.026617,-0.041210,...,-0.018067,-0.015325,0.008887,0.001714,0.013562,-0.003722,-0.017332,0.004726,0.013036,-0.066408
EMPLOYER_STATE,-0.009927,-0.019622,-0.027102,-0.110785,1.000000,-0.046735,0.055750,-0.044895,-0.048568,-0.016305,...,-0.018112,0.013116,0.149153,-0.032927,0.177240,-0.124964,0.471784,-0.001020,0.176186,-0.113825
SOC_CODE,0.042184,0.013479,0.024115,0.077274,-0.046735,1.000000,0.044641,-0.014710,-0.005389,-0.009207,...,-0.040847,-0.083424,-0.037265,-0.099187,-0.177131,0.010856,-0.005568,-0.001955,-0.179015,-0.196942
NAICS_CODE,-0.026265,-0.010785,-0.006224,0.126356,0.055750,0.044641,1.000000,-0.097621,-0.054418,-0.117184,...,-0.081892,0.025094,0.036905,-0.008031,0.206089,-0.005657,-0.007961,0.005672,0.204249,-0.024924
TOTAL_WORKERS,0.005182,0.003735,-0.004092,-0.020635,-0.044895,-0.014710,-0.097621,1.000000,0.560043,0.583499,...,0.679626,0.019541,-0.042065,0.024820,-0.047359,0.039475,-0.040002,-0.001638,-0.046952,0.056386
NEW_EMPLOYMENT,0.003979,0.094475,0.091119,0.026617,-0.048568,-0.005389,-0.054418,0.560043,1.000000,0.298523,...,0.278017,0.010938,0.007295,0.009922,-0.020845,0.022157,-0.025871,0.000773,-0.020356,-0.021527
CONTINUED_EMPLOYMENT,-0.003403,-0.066523,-0.067342,-0.041210,-0.016305,-0.009207,-0.117184,0.583499,0.298523,1.000000,...,0.597338,0.019714,-0.045869,0.019503,-0.064056,0.023894,-0.033738,-0.000697,-0.063565,0.066167


In [28]:
x2.columns

Index(['EMPLOYER_NAME', 'EMPLOYER_STATE', 'SOC_CODE', 'NAICS_CODE',
       'TOTAL_WORKERS', 'FULL_TIME_POSITION', 'H1B_DEPENDENT',
       'WILLFUL_VIOLATOR', 'TOTAL_WAGE'],
      dtype='object')

In [29]:
model=tree.DecisionTreeClassifier(criterion="entropy")
xTrain, xTest, yTrain, yTest = train_test_split(x2, y, test_size=0.3, random_state=0)

In [30]:
model.fit(xTrain,yTrain)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [31]:
model2 = model.predict(xTest)
#print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)
x2.columns

dtreeAccuracy=accuracy_score(yTest, model2)*100



confusion =confusion_matrix(yTest, model2)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
auc = roc_auc_score(yTest, model2)
specificity = TN / (TN + FP)



print('*'*10,'Dtree')
print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)
print('classification error',(1 - accuracy_score(yTest, model2))*100)
print('sensitivity:',recall_score(yTest, model2))
print('False Positive Rate:',1 - specificity)
print('AUC: %.2f' % auc)
print('Confusion Matrix')
print(confusion_matrix(yTest,model2))

The accuracy :-  98.51461804464005
********** Dtree
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    163860
           1       0.14      0.11      0.12      1552

    accuracy                           0.99    165412
   macro avg       0.56      0.55      0.56    165412
weighted avg       0.98      0.99      0.98    165412

The accuracy :-  98.51461804464005
classification error 1.4853819553599545
sensitivity: 0.11146907216494846
False Positive Rate: 0.006578786769193212
AUC: 0.55
Confusion Matrix
[[162782   1078]
 [  1379    173]]


In [32]:
model2

array([0, 0, 0, ..., 0, 0, 0])

In [34]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(xTrain, yTrain)
neigh2 = neigh.predict(xTest)
#print(classification_report(yTest, neigh2))
print('The accuracy :- ',accuracy_score(yTest, neigh2)*100)

knnAccuracy=accuracy_score(yTest, model2)*100


model2=neigh2
confusion =confusion_matrix(yTest, model2)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
auc = roc_auc_score(yTest, model2)
specificity = TN / (TN + FP)

print('*'*10,'KNN')

print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)
print('classification error',(1 - accuracy_score(yTest, model2))*100)
print('sensitivity:',recall_score(yTest, model2))
print('False Positive Rate:',1 - specificity)
print('AUC: %.2f' % auc)
print('Confusion Matrix')
print(confusion_matrix(yTest,model2))

The accuracy :-  98.9323628273644
********** KNN
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    163860
           1       0.20      0.04      0.07      1552

    accuracy                           0.99    165412
   macro avg       0.59      0.52      0.53    165412
weighted avg       0.98      0.99      0.99    165412

The accuracy :-  98.9323628273644
classification error 1.0676371726356049
sensitivity: 0.04445876288659794
False Positive Rate: 0.0017270840961797207
AUC: 0.52
Confusion Matrix
[[163577    283]
 [  1483     69]]


In [35]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(xTrain,yTrain)

model2 = rf_classifier.predict(xTest)
#print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)

rfAccuracy=accuracy_score(yTest, model2)*100




confusion =confusion_matrix(yTest, model2)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
auc = roc_auc_score(yTest, model2)
specificity = TN / (TN + FP)

print('*'*10,'Rforest')
print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)
print('classification error',(1 - accuracy_score(yTest, model2))*100)
print('sensitivity:',recall_score(yTest, model2))
print('False Positive Rate:',1 - specificity)
print('AUC: %.2f' % auc)
print('Confusion Matrix')
print(confusion_matrix(yTest,model2))

/Users/chettykulkarni/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


The accuracy :-  98.82354363649554
********** Rforest
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    163860
           1       0.22      0.10      0.14      1552

    accuracy                           0.99    165412
   macro avg       0.61      0.55      0.57    165412
weighted avg       0.98      0.99      0.99    165412

The accuracy :-  98.82354363649554
classification error 1.1764563635044634
sensitivity: 0.09987113402061856
False Positive Rate: 0.0033504210911754484
AUC: 0.55
Confusion Matrix
[[163311    549]
 [  1397    155]]


In [36]:
rf_classifier = ExtraTreeClassifier()
rf_classifier.fit(xTrain,yTrain)

model2 = rf_classifier.predict(xTest)
#print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)


confusion =confusion_matrix(yTest, model2)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
auc = roc_auc_score(yTest, model2)
specificity = TN / (TN + FP)

print('*'*10,'ExtraTree Classifier')
print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)
print('classification error',(1 - accuracy_score(yTest, model2))*100)
print('sensitivity:',recall_score(yTest, model2))
print('False Positive Rate:',1 - specificity)
print('AUC: %.2f' % auc)
print('Confusion Matrix')
print(confusion_matrix(yTest,model2))

The accuracy :-  98.50434067661354
********** ExtraTree Classifier
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    163860
           1       0.12      0.10      0.11      1552

    accuracy                           0.99    165412
   macro avg       0.56      0.55      0.55    165412
weighted avg       0.98      0.99      0.98    165412

The accuracy :-  98.50434067661354
classification error 1.4956593233864557
sensitivity: 0.09664948453608248
False Positive Rate: 0.00654217014524594
AUC: 0.55
Confusion Matrix
[[162788   1072]
 [  1402    150]]


In [37]:
rf_classifier = BernoulliNB()
rf_classifier.fit(xTrain,yTrain)

model2 = rf_classifier.predict(xTest)
#print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)

nbAccuracy=accuracy_score(yTest, model2)*100


confusion =confusion_matrix(yTest, model2)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
auc = roc_auc_score(yTest, model2)
specificity = TN / (TN + FP)

print('*'*10,'Bernouli NB')
print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)
print('classification error',(1 - accuracy_score(yTest, model2))*100)
print('sensitivity:',recall_score(yTest, model2))
print('False Positive Rate:',1 - specificity)
print('AUC: %.2f' % auc)
print('Confusion Matrix')
print(confusion_matrix(yTest,model2))

The accuracy :-  99.06234130534665
********** Bernouli NB
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    163860
           1       1.00      0.00      0.00      1552

    accuracy                           0.99    165412
   macro avg       1.00      0.50      0.50    165412
weighted avg       0.99      0.99      0.99    165412

The accuracy :-  99.06234130534665
classification error 0.9376586946533538
sensitivity: 0.0006443298969072165
False Positive Rate: 0.0
AUC: 0.50
Confusion Matrix
[[163860      0]
 [  1551      1]]


In [38]:
rf_classifier = LinearDiscriminantAnalysis()
rf_classifier.fit(xTrain,yTrain)

model2 = rf_classifier.predict(xTest)
#print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)

ldAccuracy=accuracy_score(yTest, model2)*100
confusion =confusion_matrix(yTest, model2)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
auc = roc_auc_score(yTest, model2)
specificity = TN / (TN + FP)

print(classification_report(yTest, model2))
print('The accuracy :- ',accuracy_score(yTest, model2)*100)
print('classification error',(1 - accuracy_score(yTest, model2))*100)
print('sensitivity:',recall_score(yTest, model2))
print('False Positive Rate:',1 - specificity)
print('AUC: %.2f' % auc)
print('Confusion Matrix')
print(confusion_matrix(yTest,model2))

The accuracy :-  99.06234130534665
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    163860
           1       0.67      0.00      0.00      1552

    accuracy                           0.99    165412
   macro avg       0.83      0.50      0.50    165412
weighted avg       0.99      0.99      0.99    165412

The accuracy :-  99.06234130534665
classification error 0.9376586946533538
sensitivity: 0.001288659793814433
False Positive Rate: 6.102770657934187e-06
AUC: 0.50
Confusion Matrix
[[163859      1]
 [  1550      2]]


In [41]:
print('Decision Tree Accuracy---- {}'.format(dtreeAccuracy))
print('KNN Accuracy          ---- {}'.format(knnAccuracy))
print('Naive Bayes   Accuracy---- {}'.format(nbAccuracy))
print('Random Forest Accuracy---- {}'.format(rfAccuracy))

Decision Tree Accuracy---- 98.51461804464005
KNN Accuracy          ---- 98.51461804464005
Naive Bayes   Accuracy---- 99.06234130534665
Random Forest Accuracy---- 98.82354363649554


514388    1
41583     1
565820    1
542330    1
114365    1
349159    1
2654      1
201157    1
244659    1
50389     1
395443    1
349394    1
362329    1
315729    1
351066    1
324119    1
547438    1
2655      1
441788    1
197488    1
12851     1
317294    1
389839    1
200513    1
60339     1
4078      1
624105    1
373122    1
23641     1
336068    1
         ..
12245     1
339111    1
567976    1
359702    1
77443     1
91331     1
342152    1
74652     1
2555      1
6464      1
614000    1
269485    1
506249    1
303663    1
528465    1
368900    1
355588    1
11167     1
369913    1
442875    1
501191    1
1760      1
56227     1
26311     1
345088    1
607217    1
174261    1
661624    1
335068    1
170635    1
Name: CASE_STATUS, Length: 1552, dtype: int64

In [45]:
z=x.drop(['PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'WORKSITE_COUNTY',
       'WORKSITE_STATE','EMPLOYMENT_START_DATE','EMPLOYMENT_END_DATE'], axis=1)

z=z.drop([ 'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT',
       'CHANGE_PREVIOUS_EMPLOYMENT', 'NEW_CONCURRENT_EMPLOYMENT',
       'CHANGE_EMPLOYER', 'AMENDED_PETITION','SUPPORT_H1B'], axis=1)

In [47]:
z

,CASE_STATUS,EMPLOYER_NAME,EMPLOYER_STATE,SOC_CODE,NAICS_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,H1B_DEPENDENT,WILLFUL_VIOLATOR,TOTAL_WAGE
1,1,59686,48,183,1792,1,1,0,0,3595
2,1,59686,48,240,1792,1,1,0,0,10563
3,1,7603,30,420,1932,1,1,0,0,4
4,0,12607,9,398,1827,1,1,1,0,8
5,1,8589,19,278,1790,1,1,0,0,2
6,1,24238,27,294,689,1,1,0,0,21
7,0,24238,27,294,689,1,1,0,0,21
8,1,38480,9,416,1908,10,1,0,0,9
9,1,41985,25,490,1440,1,1,0,0,39
10,0,61842,19,293,1790,1,0,0,0,28
